In [24]:
import os
import pandas as pd
from skimage.feature import hog, local_binary_pattern
from skimage.io import imread
from skimage import img_as_ubyte, transform
import numpy as np
import re

# Function to extract HOG features
def extract_hog_features(image):
    # Increase pixels_per_cell and cells_per_block to reduce the number of features
    features, hog_image = hog(image, pixels_per_cell=(32, 32), cells_per_block=(2, 2), visualize=True, multichannel=False)
    return features

# Function to extract LBP features
def extract_lbp_features(image):
    # Parameters for LBP
    radius = 3
    n_points = 8 * radius
    lbp = local_binary_pattern(image, n_points, radius, method='uniform')
    # Calculate the histogram of the LBP
    lbp_hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
    # Normalize the histogram
    lbp_hist = lbp_hist.astype("float")
    lbp_hist /= (lbp_hist.sum() + 1e-6)
    return lbp_hist

# Function to extract both HOG and LBP features
def extract_features(image_path):
    # Read image using skimage
    img = imread(image_path, as_gray=True)

    # Reduce image resolution to reduce the number of HOG features
    img_resized = transform.resize(img, (128, 128))

    # Convert image to unsigned byte (0-255)
    img_uint8 = img_as_ubyte(img_resized)

    # Extract HOG features
    hog_features = extract_hog_features(img_uint8)

    # Extract LBP features
    lbp_features = extract_lbp_features(img_uint8)

    # Combine HOG and LBP features
    combined_features = np.hstack((hog_features, lbp_features))

    return combined_features

# Directory containing your images
image_dir = '/content/drive/MyDrive/All_Leaves'

# Initialize lists to store features and corresponding labels
all_features = []
all_labels = []
all_ex_labels = []
pattern = r'C(\d+)'
pattern2 = r'(\d+)'

# Iterate over each image in the directory
for filename in os.listdir(image_dir):
    if filename.endswith('.JPG') or filename.endswith('.png'):  # Adjust file extensions as per your images
        # Extract label from filename (assuming format is 'C01.jpg', 'C02.png', etc.)
        split = filename.split('_')
        label = split[1]
        ex_label = split[2]

        label = re.findall(pattern, label)[0]
        label = int(label)

        match = re.search(pattern2, ex_label)
        if match:
            ex_label = match.group(0)
            ex_label = int(ex_label)

        # Construct image path
        image_path = os.path.join(image_dir, filename)

        # Extract features
        features = extract_features(image_path)

        # Append features and label to lists
        all_features.append(features)
        all_labels.append(label)
        all_ex_labels.append(ex_label)

# Determine the number of features dynamically
num_features = len(all_features[0])
feature_columns = [f'feature_{i}' for i in range(num_features)]

# Create a pandas DataFrame for features and labels
leaf_data = pd.DataFrame(all_features, columns=feature_columns)
leaf_data['label'] = all_labels
leaf_data['ex_label'] = all_ex_labels

# Save DataFrame to CSV
csv_file = '/content/drive/MyDrive/leaf_hog_lbp_features.csv'
leaf_data.to_csv(csv_file, index=False)

print(f'HOG and LBP features extracted and saved to {csv_file}.')


Texture features extracted and saved to /content/drive/MyDrive/leaf_texture_features.csv.
